<a href="https://colab.research.google.com/github/sunnybhakta296/LLM/blob/main/rag_with_sql_reranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG backed by SQL and Jina Reranker v2

_Authored by: [Scott Martens](https://github.com/scott-martens) @ [Jina AI](https://jina.ai)_

This notebook will show you how to make a simple Retrieval Augmented Generation (RAG) system that draws on an SQL database instead of drawing information from a document store.

### How it Works

* Given an SQL database, we extract SQL table definitions (the `CREATE` line in an SQL dump) and store them. In this tutorial, we've done this part for you and the definitions are stored in memory as a list. Scaling up from this example might require more sophisticated storage.
* The user enters a query in natural language.
* [Jina Reranker v2](https://jina.ai/reranker/) \([`jinaai/jina-reranker-v2-base-multilingual`](https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual)), an SQL-aware reranking model from [Jina AI](https://jina.ai), sorts the table definitions in order of their relevance to the user's query.
* We present [Mistral 7B Instruct v0.1 \(`mistralai/Mistral-7B-Instruct-v0.1`)](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) with a prompt containing the user's query and the top three table definitions, with a request to write an SQL query to fit the task.
* Mistral Instruct generates an SQL query and we run it against the database, retrieving a result.
* The SQL query result is converted to JSON and presented to Mistral Instruct in a new prompt, along with the user's original query, the SQL query, and a request to compose an answer for the user in natural language.
* Mistral Instruct's natural language text response is returned to the user.

### The Database

For this tutorial, we are using a small open-access database of video game sales records [stored on GitHub](https://github.com/bbrumm/databasestar/tree/main/sample_databases/sample_db_videogames/sqlite). We will be using the [SQLite](https://www.sqlite.org/index.html) version because SQLite is very compact, cross-platform, and has built-in Python support.

### Software and Hardware Requirements

We will be running the Jina Reranker v2 model locally. If you are using Google Colab to run this notebook, make sure you are using a runtime that has access to a GPU. If you are running it locally, you will need Python 3 \(this tutorial was authored using a Python 3.11 installation) and it will run *much* faster with a CUDA-enabled GPU.

We will also use the open-source [LlamaIndex RAG framework](https://www.llamaindex.ai/) extensively in this tutorial, and the [Hugging Face Inference API](https://huggingface.co/inference-api/serverless) to access Mistral 7B Instruct v0.1. You will need a [Hugging Face account](https://huggingface.co/login) and an [access token](https://huggingface.co/settings/tokens) with at least `READ` access.

> [!WARNING]
> If you are using Google Colab, SQLite is already installed. It may not be installed on your local computer.  If it's not installed, follow the instructions on the [SQLite website](https://www.sqlite.org/download.html) to install it. The Python interface code is built into Python and you don’t need to install any Python modules for it.


## Setting Up

### Install Requirements

First, install the required Python modules:

In [ ]:
!pip install -qU transformers einops llama-index llama-index-postprocessor-jinaai-rerank  llama-index-llms-huggingface "huggingface_hub[inference]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


### Download the Database

Next, download the SQLite database `videogames.db` from [GitHub](https://github.com/bbrumm/databasestar/tree/main/sample_databases/sample_db_videogames/sqlite) to the local filespace If `wget` is not available on your system, download the database from [this link](https://github.com/bbrumm/databasestar/raw/main/sample_databases/sample_db_videogames/sqlite/videogames.db) and put it in the same directory where you're running this notebook:


In [ ]:
!wget https://github.com/bbrumm/databasestar/raw/main/sample_databases/sample_db_videogames/sqlite/videogames.db

--2025-01-10 07:59:00--  https://github.com/bbrumm/databasestar/raw/main/sample_databases/sample_db_videogames/sqlite/videogames.db
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bbrumm/databasestar/main/sample_databases/sample_db_videogames/sqlite/videogames.db [following]
--2025-01-10 07:59:00--  https://raw.githubusercontent.com/bbrumm/databasestar/main/sample_databases/sample_db_videogames/sqlite/videogames.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1974272 (1.9M) [application/octet-stream]
Saving to: ‘videogames.db’

videogames.db       100%[===================>]   1.88M  --.-K

In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.2 MB/s eta 0:00:00


In [ ]:
import sqlite3
import random
from faker import Faker

conn = sqlite3.connect('employees.db')
cursor = conn.cursor()

# Create tables
cursor.executescript('''
CREATE TABLE Employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    phone_number TEXT,
    date_of_birth DATE,
    hire_date DATE NOT NULL,
    status TEXT DEFAULT 'Active',
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE Employee_Location (
    location_id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    city TEXT NOT NULL,
    state TEXT,
    country TEXT,
    zip_code TEXT,
    FOREIGN KEY (employee_id) REFERENCES Employees(employee_id)
);

CREATE TABLE Employee_Salary (
    salary_id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    salary DECIMAL(10, 2) NOT NULL,
    salary_date DATE NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES Employees(employee_id)
);

CREATE TABLE Employee_Experience (
    experience_id INTEGER PRIMARY KEY AUTOINCREMENT,
    employee_id INTEGER,
    years_of_experience INTEGER NOT NULL,
    experience_date DATE NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES Employees(employee_id)
);

CREATE TABLE Technologies (
    technology_id INTEGER PRIMARY KEY AUTOINCREMENT,
    technology_name TEXT NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE Employee_Technologies (
    employee_id INTEGER,
    technology_id INTEGER,
    proficiency_level TEXT NOT NULL,
    FOREIGN KEY (employee_id) REFERENCES Employees(employee_id),
    FOREIGN KEY (technology_id) REFERENCES Technologies(technology_id),
    PRIMARY KEY (employee_id, technology_id)
);
''')

# Insert sample data
cursor.execute('''
INSERT INTO Employees (first_name, last_name, email, phone_number, date_of_birth, hire_date, status)
VALUES ('John', 'Doe', 'john.doe@example.com', '123-456-7890', '1980-01-01', '2020-01-01', 'Active')
''')

cursor.execute('''
INSERT INTO Employee_Location (employee_id, city, state, country, zip_code)
VALUES (1, 'New York', 'NY', 'USA', '10001')
''')

cursor.execute('''
INSERT INTO Employee_Salary (employee_id, salary, salary_date)
VALUES (1, 60000.00, '2020-01-01')
''')

cursor.execute('''
INSERT INTO Employee_Experience (employee_id, years_of_experience, experience_date)
VALUES (1, 10, '2020-01-01')
''')

cursor.execute('''
INSERT INTO Technologies (technology_name)
VALUES ('Python')
''')

cursor.execute('''
INSERT INTO Employee_Technologies (employee_id, technology_id, proficiency_level)
VALUES (1, 1, 'Advanced')
''')
fake = Faker()

# Insert 100 sample employees
for _ in range(100):
    first_name = fake.first_name()
    last_name = fake.last_name()
    email = fake.email()
    phone_number = fake.phone_number()
    date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=65)
    hire_date = fake.date_this_decade()
    status = random.choice(['Active', 'Inactive', 'On Leave'])

    cursor.execute('''
    INSERT INTO Employees (first_name, last_name, email, phone_number, date_of_birth, hire_date, status)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (first_name, last_name, email, phone_number, date_of_birth, hire_date, status))

# Insert corresponding locations, salaries, experiences, and technologies for each employee
for employee_id in range(2, 102):
    city = fake.city()
    state = fake.state()
    country = fake.country()
    zip_code = fake.zipcode()

    cursor.execute('''
    INSERT INTO Employee_Location (employee_id, city, state, country, zip_code)
    VALUES (?, ?, ?, ?, ?)
    ''', (employee_id, city, state, country, zip_code))

    salary = round(random.uniform(30000, 150000), 2)
    salary_date = fake.date_this_decade()

    cursor.execute('''
    INSERT INTO Employee_Salary (employee_id, salary, salary_date)
    VALUES (?, ?, ?)
    ''', (employee_id, salary, salary_date))

    years_of_experience = random.randint(1, 40)
    experience_date = fake.date_this_decade()

    cursor.execute('''
    INSERT INTO Employee_Experience (employee_id, years_of_experience, experience_date)
    VALUES (?, ?, ?)
    ''', (employee_id, years_of_experience, experience_date))

    technology_id = random.randint(1, 1)  # Assuming only one technology exists
    proficiency_level = random.choice(['Beginner', 'Intermediate', 'Advanced'])

    cursor.execute('''
    INSERT INTO Employee_Technologies (employee_id, technology_id, proficiency_level)
    VALUES (?, ?, ?)
    ''', (employee_id, technology_id, proficiency_level))

# Commit changes and close connection
conn.commit()
conn.close()


### Download and Run Jina Reranker v2

The following code will download the model `jina-reranker-v2-base-multilingual` and run it locally:


In [ ]:
from transformers import AutoModelForSequenceClassification

reranker_model = AutoModelForSequenceClassification.from_pretrained(
    'jinaai/jina-reranker-v2-base-multilingual',
    torch_dtype="auto",
    trust_remote_code=True,
)

reranker_model.to('cuda') # or 'cpu' if no GPU is available
reranker_model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

configuration_xlm_roberta.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- configuration_xlm_roberta.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_xlm_roberta.py:   0%|          | 0.00/43.8k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- mha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- block.py
- mha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


embedding.py:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual:
- modeling_xlm_roberta.py
- xlm_padding.py
- mlp.py
- block.py
- embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768)
      (token_type_embeddings): Embedding(1, 768)
    )
    (emb_drop): Dropout(p=0.1, inplace=False)
    (emb_ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder): XLMRobertaEncoder(
      (layers): ModuleList(
        (0-11): 12 x Block(
          (mixer): MHA(
            (Wqkv): LinearResidual(in_features=768, out_features=2304, bias=True)
            (inner_attn): SelfAttention(
              (drop): Dropout(p=0.1, inplace=False)
            )
            (inner_cross_attn): CrossAttention(
              (drop): Dropout(p=0.1, inplace=False)
            )
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (dropout1): Dropout(p=0.1, inplace=False)
          (drop_path1): StochasticDepth(p=0.0, mode=r

### Set up the Interface to Mistral Instruct

We will use LlamaIndex to create a holder object for the connection to the Hugging Face inference API and to the copy of `mistralai/Mixtral-8x7B-Instruct-v0.1` running there.


First, get a Hugging Face access token from your [Hugging Face Account Settings page](https://huggingface.co/settings/tokens).

Enter it when prompted below:

In [ ]:
import getpass

print("Paste your Hugging Face access token here: ")
hf_token = getpass.getpass()

Paste your Hugging Face access token here: 
··········


Next, initialize an instance of the `HuggingFaceInferenceAPI` class from LlamaIndex and store it as `mistral_llm`:

In [ ]:
from llama_index.llms.huggingface import HuggingFaceInferenceAPI

mistral_llm = HuggingFaceInferenceAPI(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1", token=hf_token
)

<ipython-input-9-2aa73ccaea1f>:3: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  mistral_llm = HuggingFaceInferenceAPI(


## Using SQL-Aware Jina Reranker v2

We extracted the eight table definitions from the [database import files located on GitHub](https://github.com/bbrumm/databasestar/tree/main/sample_databases/sample_db_videogames/sqlite). Run the command below to put them into a Python list named `table_declarations`:

In [ ]:
table_declarations = [
    'CREATE TABLE Employees (\n\temployee_id INT PRIMARY KEY AUTO_INCREMENT,\n\tfirst_name VARCHAR(100) NOT NULL,\n\tlast_name VARCHAR(100) NOT NULL,\n\temail VARCHAR(255) UNIQUE NOT NULL,\n\tphone_number VARCHAR(20),\n\tdate_of_birth DATE,\n\thire_date DATE NOT NULL,\n\tstatus ENUM(\'Active\', \'Inactive\', \'On Leave\') DEFAULT \'Active\',\n\tcreated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n\tupdated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP\n);',
    'CREATE TABLE Employee_Location (\n\tlocation_id INT PRIMARY KEY AUTO_INCREMENT,\n\temployee_id INT,\n\tcity VARCHAR(255) NOT NULL,\n\tstate VARCHAR(255),\n\tcountry VARCHAR(255),\n\tzip_code VARCHAR(20),\n\tFOREIGN KEY (employee_id) REFERENCES Employees(employee_id)\n);',
    'CREATE TABLE Employee_Salary (\n\tsalary_id INT PRIMARY KEY AUTO_INCREMENT,\n\temployee_id INT,\n\tsalary DECIMAL(10, 2) NOT NULL,\n\tsalary_date DATE NOT NULL,\n\tFOREIGN KEY (employee_id) REFERENCES Employees(employee_id)\n);',
    'CREATE TABLE Employee_Experience (\n\texperience_id INT PRIMARY KEY AUTO_INCREMENT,\n\temployee_id INT,\n\tyears_of_experience INT NOT NULL,\n\texperience_date DATE NOT NULL,\n\tFOREIGN KEY (employee_id) REFERENCES Employees(employee_id)\n);',
    'CREATE TABLE Technologies (\n\ttechnology_id INT PRIMARY KEY AUTO_INCREMENT,\n\ttechnology_name VARCHAR(255) NOT NULL,\n\tcreated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP\n);',
    'CREATE TABLE Employee_Technologies (\n\temployee_id INT,\n\ttechnology_id INT,\n\tproficiency_level ENUM(\'Beginner\', \'Intermediate\', \'Advanced\') NOT NULL,\n\tFOREIGN KEY (employee_id) REFERENCES Employees(employee_id),\n\tFOREIGN KEY (technology_id) REFERENCES Technologies(technology_id),\n\tPRIMARY KEY (employee_id, technology_id)\n);'
]

Now, we define a function that takes a natural language query and the list of table definitions, scores all of them with Jina Reranker v2, returning them in order from highest scoring to lowest:

In [ ]:
from typing import List, Tuple

def rank_tables(query: str, table_specs: List[str], top_n:int=0) -> List[Tuple[float, str]]:
  """
  Get sorted pairs of scores and table specifications, then return the top N,
  or all if top_n is 0 or default.
  """
  pairs = [[query, table_spec] for table_spec in table_specs]
  scores = reranker_model.compute_score(pairs)
  scored_tables = [(score, table_spec) for score, table_spec in zip(scores, table_specs)]
  scored_tables.sort(key=lambda x: x[0], reverse=True)
  if top_n and top_n < len(scored_tables):
    return scored_tables[0:top_n]
  return scored_tables

Jina Reranker v2 scores every table definition we give it and by default this function will return all of them with their scores. The optional argument `top_n` limits the number of results returned to a user-defined number, starting with the highest scoring one.

Try it out. First, define a query:

In [ ]:
user_query = "find emloyee where email is john.doe@example.com"
user_query

'find emloyee where email is john.doe@example.com'

Run `rank_tables` to get a list of table definitions back. Let's set `top_n` to 3 to limit the return list size and assign it to the variable `ranked_tables`, then inspect the result:

In [ ]:
ranked_tables = rank_tables(user_query, table_declarations, top_n=10)
ranked_tables

[(0.13296423852443695,
  'CREATE TABLE Employee_Location (\n\tlocation_id INT PRIMARY KEY AUTO_INCREMENT,\n\temployee_id INT,\n\tcity VARCHAR(255) NOT NULL,\n\tstate VARCHAR(255),\n\tcountry VARCHAR(255),\n\tzip_code VARCHAR(20),\n\tFOREIGN KEY (employee_id) REFERENCES Employees(employee_id)\n);'),
 (0.10521054267883301,
  "CREATE TABLE Employees (\n\temployee_id INT PRIMARY KEY AUTO_INCREMENT,\n\tfirst_name VARCHAR(100) NOT NULL,\n\tlast_name VARCHAR(100) NOT NULL,\n\temail VARCHAR(255) UNIQUE NOT NULL,\n\tphone_number VARCHAR(20),\n\tdate_of_birth DATE,\n\thire_date DATE NOT NULL,\n\tstatus ENUM('Active', 'Inactive', 'On Leave') DEFAULT 'Active',\n\tcreated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n\tupdated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP\n);"),
 (0.05834583938121796,
  'CREATE TABLE Employee_Experience (\n\texperience_id INT PRIMARY KEY AUTO_INCREMENT,\n\temployee_id INT,\n\tyears_of_experience INT NOT NULL,\n\texperience_date DATE NOT NULL,\n\tFOR

The output should include the tables `region_sales`, `platform` and `game_platform`, which all seem to be reasonable places to look for an answer to the query.

## Using Mistral Instruct to Generate SQL

We're going to have Mistral Instruct v0.1 write an SQL query that fulfils the user's query, based on the declarations of the top three tables according to the reranker.

First, we make a prompt for that purpose using LlamaIndex' `PromptTemplate` class:

In [ ]:
from llama_index.core import PromptTemplate

make_sql_prompt_tmpl_text = (
    """
Generate a SQL query to answer the following question from the user:
\"{query_str}\"

The SQL query should use only tables with the following SQL definitions:

Table 1:
{table_1}

Table 2:
{table_2}

Table 3:
{table_3}

Table 4:
{table_4}

Table 5:
{table_5}

Table 6:
{table_6}

Make sure you ONLY output an SQL query and no explanation.
"""
)
make_sql_prompt_tmpl = PromptTemplate(make_sql_prompt_tmpl_text)

We use the `format` method to fill in the template fields with the user query and top three table declarations from Jina Reranker v2:

In [ ]:
make_sql_prompt = make_sql_prompt_tmpl.format(query_str="List the employess Living in France",
                                              table_1=ranked_tables[0][1],
                                              table_2=ranked_tables[1][1],
                                              table_3=ranked_tables[2][1],
                                              table_4=ranked_tables[3][1],
                                              table_5=ranked_tables[4][1],
                                              table_6=ranked_tables[5][1])

You can see the actual text we're going to pass to Mistral Instruct:

In [ ]:
print(make_sql_prompt)


Generate a SQL query to answer the following question from the user:
"List the employess Living in France"

The SQL query should use only tables with the following SQL definitions:

Table 1:
CREATE TABLE Employee_Location (
	location_id INT PRIMARY KEY AUTO_INCREMENT,
	employee_id INT,
	city VARCHAR(255) NOT NULL,
	state VARCHAR(255),
	country VARCHAR(255),
	zip_code VARCHAR(20),
	FOREIGN KEY (employee_id) REFERENCES Employees(employee_id)
);

Table 2:
CREATE TABLE Employees (
	employee_id INT PRIMARY KEY AUTO_INCREMENT,
	first_name VARCHAR(100) NOT NULL,
	last_name VARCHAR(100) NOT NULL,
	email VARCHAR(255) UNIQUE NOT NULL,
	phone_number VARCHAR(20),
	date_of_birth DATE,
	hire_date DATE NOT NULL,
	status ENUM('Active', 'Inactive', 'On Leave') DEFAULT 'Active',
	created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
	updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
);

Table 3:
CREATE TABLE Employee_Experience (
	experience_id INT PRIMARY KEY AUTO_INCREMENT,
	employ

Now let's send the prompt to Mistral Instruct and retrieve its response:

In [ ]:
response = mistral_llm.complete(make_sql_prompt)
sql_query = str(response)
sql_query = sql_query.replace("\\", "")
print(sql_query)


SELECT Employees.first_name, Employees.last_name, Employee_Location.city, Employee_Location.country
FROM Employees
JOIN Employee_Location ON Employees.employee_id = Employee_Location.employee_id
WHERE Employee_Location.country = 'France'
ORDER BY Employees.last_name ASC;


## Running the SQL query

Use the built-in Python interface to SQLite to run the query above
against the database `videogames.db`:

In [ ]:
import sqlite3

con = sqlite3.connect("employees.db")
cur = con.cursor()
sql_response = cur.execute(sql_query).fetchall()

For details on the interface to SQLite, [see the Python3 documentation](https://docs.python.org/3/library/sqlite3.html).

Inspect the result:

In [ ]:
sql_response

[(1,
  'John',
  'Doe',
  'john.doe@example.com',
  '123-456-7890',
  '1980-01-01',
  '2020-01-01',
  'Active',
  '2025-01-10 08:33:00',
  '2025-01-10 08:33:00')]

You can check if this is correct by running your own SQL query. The sales data stored in this database is in the form of floating point numbers, presumably thousands or millions of unit sales.

## Getting a Natural Language Answer

Now we will pass the user's query, the SQL query, and the result back to Mistral Instruct with a new prompt template.

First, make the new prompt template using LlamaIndex, the same as above:

In [ ]:
rag_prompt_tmpl_str = (
    """
Use the information in the JSON table to answer the following user query.
Do not explain anything, just answer concisely. Use natural language in your
answer, not computer formatting.

USER QUERY: {query_str}

JSON table:
{json_table}

This table was generated by the following SQL query:
{sql_query}

Answer ONLY using the information in the table and the SQL query, and if the
table does not provide the information to answer the question, answer
"No Information".
"""
)
rag_prompt_tmpl = PromptTemplate(rag_prompt_tmpl_str)

We will convert the SQL output into JSON, a format Mistral Instruct v0.1
understands.

Populate the template fields:

In [ ]:
import json

rag_prompt = rag_prompt_tmpl.format(query_str="find emloyee where email is john.doe@example.com",
                                    json_table=json.dumps(sql_response),
                                    sql_query=sql_query)

Now solicit a natural language response from Mistral Instruct:

In [ ]:
rag_response = mistral_llm.complete(rag_prompt)
print(str(rag_response))


Answer:

The employee with email john.doe@example.com is John Doe.


## Try it yourself

Let's organize all that into one function with exception trapping:

In [ ]:
def answer_sql(user_query: str) -> str:
  try:
    ranked_tables = rank_tables(user_query, table_declarations, top_n=10)
  except Exception as e:
    print(f"Ranking failed.\nUser query:\n{user_query}\n\n")
    raise(e)

  make_sql_prompt = make_sql_prompt_tmpl.format(query_str=user_query,
                                                table_1=ranked_tables[0][1],
                                                table_2=ranked_tables[1][1],
                                                table_3=ranked_tables[2][1],
                                                table_4=ranked_tables[3][1],
                                                table_5=ranked_tables[4][1],
                                                table_6=ranked_tables[5][1])

  try:
    response = mistral_llm.complete(make_sql_prompt)
  except Exception as e:
    print(f"SQL query generation failed\nPrompt:\n{make_sql_prompt}\n\n")
    raise(e)

  # Backslash removal is a necessary hack because sometimes Mistral puts them
  # in its generated code.
  sql_query = str(response).replace("\\", "")
  print("---------------------------")
  print(sql_query)
  try:
    sql_response = sqlite3.connect("employees.db").cursor().execute(sql_query).fetchall()
  except Exception as e:
    print(f"SQL querying failed. Query:\n{sql_query}\n\n")
    raise(e)

  rag_prompt = rag_prompt_tmpl.format(query_str=user_query,
                                      json_table=json.dumps(sql_response),
                                      sql_query=sql_query)
  try:
    rag_response = mistral_llm.complete(rag_prompt)
    return str(rag_response)
  except Exception as e:
    print(f"Answer generation failed. Prompt:\n{rag_prompt}\n\n")
    raise(e)

Try it out:

In [ ]:
print(answer_sql("Find employee details with salary less then 100000"))

---------------------------

SELECT Employees.first_name, Employees.last_name, Employee_Salary.salary
FROM Employees
JOIN Employee_Salary ON Employees.employee_id = Employee_Salary.employee_id
WHERE Employee_Salary.salary < 100000;

1. What is the name of the employee with the lowest salary?

Derek Bautista


Try some other queries:

In [ ]:
print(answer_sql("Summarize employees by status."))

In [ ]:
print(answer_sql("List all the employess From New York state"))

---------------------------

SELECT Employees.first_name, Employees.last_name, Employee_Location.city, Employee_Location.state
FROM Employees
JOIN Employee_Location
ON Employees.employee_id = Employee_Location.employee_id
WHERE Employee_Location.state = 'New York';

List all employees from New York state:

Jacob Padilla
Stephen Brown
Amber Church
Marvin Lewis
Amy Hutchinson


In [ ]:
print(answer_sql("Display the year with most games released."))

In [ ]:
print(answer_sql("What is the most popular game genre on the Wii platform?"))

In [ ]:
print(answer_sql("What is the most popular game genre of 2012?"))

Try your own queries:


In [ ]:
print(answer_sql("<INSERT QUESTION OR INSTRUCTION HERE>"))

## Review and Conclusions

We've shown you how to make a very basic RAG (retrieval-augmented generation) system for natural language question-answering that uses an SQL database as an information source.  In this implementation, we use the same large language model (Mistral Instruct v0.1), to generate SQL queries and to construct natural language responses.

The database here is a very small example, and scaling this up might demand a more sophisticated approach than just ranking a list of table definitions. You might want to use a two-stage process, where an embedding model and vector store initially retrieve more results, but the reranker model prunes that down to whatever number you are able to put into a prompt for a generative language model.

This notebook has assumed no request requires more than three tables to satisfy, and obviously, in practice, this cannot always be true. Mistral 7B Instruct v0.1 is not guaranteed to produce correct (or even executable) SQL output. In production, something like this requires much more in-depth error handling.

More sophisticated error handling, longer input context windows, and generative models specialized in SQL-specific tasks might make a big difference in practical applications.

Nonetheless, you can see here how the RAG concept extends to structured databases, expanding its scope for use dramatically.